In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
from astropy import units
import os
import astropy.units as u
import matplotlib.pyplot as plt

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Further tests of the different distinctions to satellites and isolated galaxies

## Notes
From the definition of satellites by distance (call this set of subhalos SD) and the definition of satellites by their group numbers and subgroup numbers (call this set of subhalos SGN), the following statements follow:
* Any subhalo in SGN that also lies within 300 kpc of a central galaxy will also be identified as a satellite in SD
* No subhalo in SD lies beyond 300 kpc from a central, but the same is not true of subhalos in SGN
* Subhalos that lie in further than 300 kpc from any central but within 2 Mpc of the LG centre, will either be an element of both SD and SGN or neither (depending on their subgroup number)

Other general remarks about what one would expect to find, if the distinction methods have been implemented correctly:
* From the last statement above, one expects identification in the range $500 \mathrm{kpc} \lesssim r_\mathrm{LG} < 2 \mathrm{Mpc}$ to be almost identical in both distinctions

## Tests
Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

## Get data for plotting

Choose how to distinguish between satellites and isolated:

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
def arrange_radius_array(r):
    r = r * units.cm.to(units.kpc)
    r = np.concatenate((np.sort(r), np.array([10000]))) 
    return r

def construct_count_array(size):
    return np.concatenate((np.arange(1, size+1), 
                           np.array([size])))

In [ ]:
for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential")
    
    # Compute masking arrays:
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    masks_sat_by_r, mask_isol_by_r = dataset_compute.split_satellites_by_distance(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    masks_sat_by_gn, mask_isol_by_gn = dataset_compute.split_satellites_by_group_number(
        snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    
    mask_m31_by_r = np.logical_and(masks_sat_by_r[0], mask_nonzero_vmax)
    mask_mw_by_r = np.logical_and(masks_sat_by_r[1], mask_nonzero_vmax)
    mask_isol_by_r = np.logical_and(mask_isol_by_r, mask_nonzero_vmax)
    mask_m31_by_gn = np.logical_and(masks_sat_by_gn[0], mask_nonzero_vmax)
    mask_mw_by_gn = np.logical_and(masks_sat_by_gn[1], mask_nonzero_vmax)
    mask_isol_by_gn = np.logical_and(mask_isol_by_gn, mask_nonzero_vmax)
    print(np.sum(mask_nonzero_vmax[:200]))
    print(np.sum(mask_m31_by_gn[:100]))
    print(np.sum(mask_mw_by_gn[:140]))
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_mw_by_r = np.logical_and(mask_mw_by_r, np.logical_not(mask_m31_by_r))
    mask_mw_by_gn = np.logical_and(mask_mw_by_gn, np.logical_not(mask_m31_by_gn))
    
    # Some sanity checks:
    gns = snap.get_subhalos("GroupNumber")
    sgns = snap.get_subhalos("SubGroupNumber")
    print(np.bincount(gns.astype(int)))
    print(np.bincount(gns[
        np.logical_and(np.logical_or(mask_m31_by_r, mask_mw_by_r), sgns == 0)
    ].astype(int)))
   
    # Compute distances to M31, MW and the LG centre:
    m31_cop = cops[snap.index_of_halo(sim_data["M31_identifier"][0],
                                      sim_data["M31_identifier"][1])]
    dist_to_m31 = dataset_compute.distance_to_point(snap, m31_cop)
    mw_cop = cops[snap.index_of_halo(sim_data["MW_identifier"][0],
                                     sim_data["MW_identifier"][1])]
    dist_to_mw = dataset_compute.distance_to_point(snap, mw_cop)    
    lg_centre = dataset_compute.compute_LG_centre(snap,
                                                  sim_data["M31_identifier"],
                                                  sim_data["MW_identifier"])
    dist_to_lg = dataset_compute.distance_to_point(snap, lg_centre)


    # Add satellites to the data dictionary:
    sat_dist_by_r = arrange_radius_array(
        np.concatenate([dist_to_m31[mask_m31_by_r], dist_to_mw[mask_mw_by_r]]))
    sat_dist_by_gn = arrange_radius_array(
        np.concatenate([dist_to_m31[mask_m31_by_gn], dist_to_mw[mask_mw_by_gn]]))
    
    sim_data["satellites"] = \
    {"radius": {"by_distance": sat_dist_by_r,
                "by_groupnumber": sat_dist_by_gn
               },
     "sat_count": {"by_distance": construct_count_array(
         np.sum(np.logical_or(mask_m31_by_r, mask_mw_by_r))),
                   "by_groupnumber": construct_count_array(
         np.sum(np.logical_or(mask_m31_by_gn, mask_mw_by_gn)))
                  }
    }

    # Add isolated galaxies to the data dictionary:
    isol_dist_by_r = arrange_radius_array(dist_to_lg[mask_isol_by_r])
    isol_dist_by_gn = arrange_radius_array(dist_to_lg[mask_isol_by_gn])
    
    sim_data["isolated"] = \
    {"radius": {"by_distance": isol_dist_by_r,
                "by_groupnumber": isol_dist_by_gn
               },
     "sat_count": {"by_distance": construct_count_array(np.sum(mask_isol_by_r)),
                   "by_groupnumber": construct_count_array(np.sum(mask_isol_by_gn))
                  }
    }


## Plot

In [ ]:
# Set some parameters:
x_down_sat = 0; x_up_sat = 1000
y_down_sat = 1; y_up_sat = 500

x_down_isol = 0; x_up_isol = 2100
y_down_isol = 1; y_up_isol = 500

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'compare_distinctions_radial_dist'
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis for satellites:
axes[0].set_xlim(x_down_sat, x_up_sat)
axes[0].set_ylim(y_down_sat, y_up_sat)
axes[0].set_xlabel('$r_\mathrm{central}[\mathrm{kpc}]$', fontsize=16)
axes[0].set_ylabel('$N(<r_\mathrm{central})$', fontsize=16)
axes[0].set_yscale('log')
axes[0].yaxis.set_ticks_position("both")

# Set axis for isolated galaxies:
axes[1].set_xlim(x_down_isol, x_up_isol)
axes[1].set_ylim(y_down_isol, y_up_isol)
axes[1].set_xlabel('$r_\mathrm{LG}}[\mathrm{kpc}]$', fontsize=16)
axes[1].set_ylabel('$N(<r_\mathrm{LG})$', fontsize=16)
axes[1].set_yscale('log')
axes[1].yaxis.set_ticks_position("both")
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['satellites']['radius']["by_distance"]
    y = entry['satellites']['sat_count']["by_distance"]
    axes[0].plot(x, y, c=color[i], label='{} by distance'.format(name))
    x = entry['satellites']['radius']["by_groupnumber"]
    y = entry['satellites']['sat_count']["by_groupnumber"]
    axes[0].plot(x, y, c=color[i], linestyle='--', label='{} by group number'.format(name))
    
    x = entry['isolated']['radius']["by_distance"]
    y = entry['isolated']['sat_count']["by_distance"]
    axes[1].plot(x, y, c=color[i], label='{} by distance'.format(name))
    x = entry['isolated']['radius']["by_groupnumber"]
    y = entry['isolated']['sat_count']["by_groupnumber"]
    axes[1].plot(x, y, c=color[i], linestyle='--', label='{} by group number'.format(name))

axes[0].legend(loc='upper left')
plt.tight_layout()

#plt.savefig(filename, dpi=200)

## Conclusions

There are no inconsistensies with the above given statements. The curious gap between the two p082 lines is due to the fact that, what most reasonably is identified as the MW galaxy in this simulation, is in fact a subgroup of the FoF halo of the M31 galaxy. In the case of the dashed line, there is only one radius $r_\mathrm{central}$, which is the distance from the M31 galaxy. In the case of the solid line, $r_\mathrm{central}$ is defined as the distance to the central closest to them. Thus, the boost in the dashed line in the region $500 \lesssim r_\mathrm{central} \lesssim 800$, is transferred to the region below 300 kpc in the solid line.